## Imports

In [25]:
import sys
sys.path.append('..')
import numpy as np
from MAC.tests.random_test import RandomAgent
from MAC.control import controller_decentralized
from MAC.control.controller_economic import EconomicControl
from MAC.agents import agent, policy_gradient

### Setting Up Taxi

In [17]:
sys.path.append('../environments/MultiTaxiEnv')
from MAC.environments.MultiTaxiEnv.taxi_environment import TaxiEnv
def get_taxi_env():
    env = TaxiEnv(2)
    # Make sure it works with our API:
    env.agents = env.taxis_names
    env.action_spaces = {
        agent_name: env.action_space for agent_name in env.agents
    }
    env.observation_spaces = {
        agent_name: env.observation_space for agent_name in env.agents
    }
    env.possible_agents = [agent for agent in env.agents]
    return env

### Decentralized with random agents 

In [18]:
env = get_taxi_env()
agents = {agent_name: RandomAgent(env.action_spaces[agent_name])
              for agent_name in env.agents}
    
control = controller_decentralized.Decentralized(agents, env)
control.run(render=True, max_iteration=100)

+---------+
|X: |F: :X|
| : | : : |
| : : : : |
| | : | : |
|X| :G|X: |
+---------+
Taxi1-YELLOW: Fuel: 100, Location: (4,2), Collided: False
Taxi2-RED: Fuel: 100, Location: (4,0), Collided: False
Passenger1: Location: (4, 0), Destination: (0, 0)
Done: False, {'taxi_1': False, 'taxi_2': False, '__all__': False}
Passengers Status's: [2]
+---------+
|X: |F: :X|
| : | : : |
| : : : : |
| | : | : |
|X| :G|X: |
+---------+
  (dropoff ,south)
Taxi1-YELLOW: Fuel: 100, Location: (4,2), Collided: False
Taxi2-RED: Fuel: 100, Location: (4,0), Collided: False
Passenger1: Location: (4, 0), Destination: (0, 0)
Done: False, {'taxi_1': False, 'taxi_2': False, '__all__': False}
Passengers Status's: [2]
+---------+
|X: |F: :X|
| : | : : |
| : : : : |
| | : | : |
|X| :G|X: |
+---------+
  (pickup ,west)
Taxi1-YELLOW: Fuel: 100, Location: (4,2), Collided: False
Taxi2-RED: Fuel: 100, Location: (4,0), Collided: False
Passenger1: Location: (4, 0), Destination: (0, 0)
Done: False, {'taxi_1': False, 'taxi_2': 

### Economic Control 

In [19]:
sys.path.append('environments/corners')
from corners_env import CornersEnv
from corners_agents import EconomicGoalAgent
env = CornersEnv()

tasks = [[0,0], [0, 4], [4, 0], [4, 4], [2,2]] # The goal position

In [20]:
agents = {agent: EconomicGoalAgent() for agent in env.agents}
controller = EconomicControl(env, agents, tasks)
controller.run(float('inf'))

Step 1:
╔═══════════╗
║ D         ║
║     E   C ║
║           ║
║ B         ║
║         A ║
╚═══════════╝

Step 2:
╔═══════════╗
║ D       C ║
║           ║
║     E     ║
║           ║
║ B       A ║
╚═══════════╝

Finished


### PG Algorithm

In [26]:
env = get_taxi_env()
num_actions_per_agent = env.action_spaces[env.possible_agents[0]].n

if len(env.observation_spaces[env.possible_agents[0]].shape) == 2:
    obs_size_per_agent = env.observation_spaces[env.possible_agents[0]].shape[1]
else:
    obs_size_per_agent = env.observation_spaces[env.possible_agents[0]].shape[0]

agents = {agent_name: agent.Agent(policy_gradient.PolicyGradient(num_actions_per_agent,
          np.random.rand(obs_size_per_agent, num_actions_per_agent),mapping_fn=lambda x: x.flatten()))
          for agent_name in env.possible_agents}
control = controller_decentralized.Decentralized(agents, env)
control.train(25, 100)
control.run(render=True, max_iteration=100)

Starting training...
+---------+
|X: |F: :X|
|_: | : : |
| : : : : |
| | : | : |
|X| :G|X: |
+---------+
Taxi1-YELLOW: Fuel: 100, Location: (4,2), Collided: False
Taxi2-RED: Fuel: 100, Location: (1,0), Collided: False
Passenger1: Location: (4, 0), Destination: (0, 4)
Done: False, {'taxi_1': False, 'taxi_2': False, '__all__': False}
Passengers Status's: [2]
+---------+
|X: |F: :X|
|_: | : : |
| : : : : |
| | : | : |
|X| :G|X: |
+---------+
  (east ,refuel)
Taxi1-YELLOW: Fuel: 100, Location: (4,2), Collided: False
Taxi2-RED: Fuel: 100, Location: (1,0), Collided: False
Passenger1: Location: (4, 0), Destination: (0, 4)
Done: False, {'taxi_1': False, 'taxi_2': False, '__all__': False}
Passengers Status's: [2]
+---------+
|X: |F: :X|
|_: | : : |
| : : : : |
| | : | : |
|X| :G|X: |
+---------+
  (pickup ,refuel)
Taxi1-YELLOW: Fuel: 100, Location: (4,2), Collided: False
Taxi2-RED: Fuel: 100, Location: (1,0), Collided: False
Passenger1: Location: (4, 0), Destination: (0, 4)
Done: False, {'taxi_